In [1]:
import os
import queue
import cv2
import numpy as np
from PIL import Image, ImageDraw
import csv
import sys

In [2]:
vids = ["Error1Raw.mp4", "Error2Raw.mp4", "InPlayBalls_54min_vid.mp4"]

In [3]:
def get_trajectory(predict_csv_dir, vid_dir, trajectory_dir):
    # read in predictions
    with open(predict_csv_dir) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        frames = []
        x, y = [], []
        list1 = []
        for row in readCSV:
            list1.append(row)
        for i in range(1 , len(list1)):
            frames += [int(list1[i][0])]
            x += [int(float(list1[i][2]))]
            y += [int(float(list1[i][3]))]
    
    num_preds = len(x)
    # make queue to draw trajectory
    q = queue.deque()
    for i in range(0,8):
        q.appendleft(None)

    # get video fps&video size
    currentFrame= 0
    video = cv2.VideoCapture(vid_dir)
    fps = int(video.get(cv2.CAP_PROP_FPS))
    num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    output_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    output_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video = cv2.VideoWriter(trajectory_dir, fourcc, fps, (output_width,output_height))
    video.set(1,currentFrame)

    print(f"\tnum_frames: {num_frames}, num_preds: {num_preds}")

    ret, img1 = video.read()
    #write image to video
    output_video.write(img1)
    currentFrame +=1
    #input must be float type
    img1 = img1.astype(np.float32)
    #capture frame-by-frame
    video.set(1,currentFrame);
    ret, img = video.read()
    #write image to video
    output_video.write(img)
    currentFrame +=1
    #input must be float type
    img = img.astype(np.float32)
    while(currentFrame < num_preds and currentFrame < num_frames):
        #capture frame-by-frame
        video.set(1,currentFrame); 
        ret, img = video.read()
            #if there dont have any frame in video, break
        if not ret: 
            break
        PIL_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   
        PIL_image = Image.fromarray(PIL_image)
        if x[currentFrame] != 0 and y[currentFrame] != 0:
            q.appendleft([x[currentFrame],y[currentFrame]])
            q.pop()
        else:
            q.appendleft(None)
            q.pop()
        for i in range(0,8):
            if q[i] is not None:
                draw_x = q[i][0]
                draw_y = q[i][1]
                bbox =  (draw_x - 2, draw_y - 2, draw_x + 2, draw_y + 2)
                draw = ImageDraw.Draw(PIL_image)
                draw.ellipse(bbox, outline ='yellow')
                del draw
        opencvImage =  cv2.cvtColor(np.array(PIL_image), cv2.COLOR_RGB2BGR)
        #write image to output_video
        output_video.write(opencvImage)
        #next frame
        currentFrame += 1
    video.release()
    output_video.release()
    print(f"\tFinished on frame {currentFrame}")
    

In [7]:
# versions = ["old", "new"]
versions = ["new"]

In [8]:
for vid in vids[2:3]:
    vid_root = vid[:-4]
    vid_dir = f"{vid_root}/{vid}"
    for version in versions:
        predict_csv_dir = f"{vid_root}/{vid_root}_{version}_weights_predict.csv"
        trajectory_dir = f"{vid_root}/{vid_root}_{version}_weights_trajectory.mp4"
        print(f"Getting {vid} trajectory with {version} weights...")
        get_trajectory(predict_csv_dir, vid_dir, trajectory_dir)

Getting InPlayBalls_54min_vid.mp4 trajectory with new weights...
	num_frames: 12810, num_preds: 12810
	Finished on frame 12810
